In [1]:
import pygame
import numpy as np
pygame.init()  

pygame 2.3.0 (SDL 2.24.2, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


(5, 0)

In [2]:
class Tela(object):
    def __init__(self, title, bgColor, width, height):
        self.title = title      
        self.bgColor = bgColor  
        self.width = width      
        self.height = height    
        self.tela = pygame.display.set_mode(self.obterTamanho()) 
        pygame.display.set_caption(self.title)             
        
    def executar(self, obj):
        
        self.tela.fill(self.bgColor)
            
        obj.draw(self);
        while True:  
            for event in pygame.event.get(): 
                if event.type == pygame.QUIT:
                    return pygame.quit()

            pygame.display.update()
        
    def obterTamanho(self):
        return (self.width, self.height)
    
    def definirPixel(self, x, y, color):
        self.tela.set_at((x, y), color)

In [3]:
class Line(object):
    def __init__(self, x1, y1, x2, y2, color):
        self.x1 = x1      
        self.x2 = x2      
        self.y1 = y1      
        self.y2 = y2       
        self.color = color 
    
    def draw(self, tela):
        self.dda(tela)

    def dda(self, tela):      
        dx, dy, k = 0, 0, 0
        x_inc, y_inc = 0.0, 0.0
        x, y = 0.0, 0.0
    
        dx = self.x2 - self.x1
        dy = self.y2 - self.y1
    
        if abs(dx) > abs(dy):
            iter = abs(dx)
        else:
            iter = abs(dy)
        
        x_inc = dx/iter
        y_inc = dy/iter

        x = self.x1
        y = self.y1

        tela.definirPixel(round(x), round(y), self.color)

        for k in range(iter):
            x = x + x_inc
            y = y + y_inc
        
            tela.definirPixel(round(x), round(y), self.color)

In [4]:
class Point2D(object):
    def __init__(self, x, y, color):
        self.x = x
        self.y = y
        self.color = color
    
    def draw(self, tela):
        tela.definirPixel(self.x, self.y, self.color)
        
    @property
    def x(self):
        return self._x
    
    @x.setter
    def x(self, x):
        self._x = x
    
    @property
    def y(self):
        return self._y
    
    @y.setter
    def y(self, y):
        self._y = y

In [5]:
class EdgeInfo(object):
    def __init__(self, initialPoint, finalPoint):
        if initialPoint.y <= finalPoint.y:
            self.yMax = finalPoint.y
            self.x = initialPoint.x  
            self.yMin = initialPoint.y
        else:
            self.yMax = initialPoint.y
            self.x = finalPoint.x    
            self.yMin = finalPoint.y
            
        self.inverseOfAngularCoefficient = (finalPoint.x - initialPoint.x) \
                                            / (finalPoint.y - initialPoint.y)
            
    @property
    def yMax(self):
        return self._yMax
    
    @yMax.setter
    def yMax(self, yMax):
        self._yMax = yMax
    
    @property
    def yMin(self):
        return self._yMin
    
    @yMin.setter
    def yMin(self, yMin):
        self._yMin = yMin
    
    @property
    def x(self):
        return self._x
    
    @x.setter
    def x(self, x):
        self._x = x
    
    def updateX(self):
        self.x = self.x + self.inverseOfAngularCoefficient
    
    def __str__(self):
        return f'yMin: {self.yMin} | yMax: {self.yMax} | x: {self.x} | coef {self.inverseOfAngularCoefficient}'

In [6]:
class Polygon(object):
    def __init__(self, showEdges, edgeColor, isFilled, fillColor):
        self.listOfPoints = []
        self.showEdges = showEdges
        self.edgeColor = edgeColor
        self.isFilled = isFilled
        self.fillColor = fillColor
    
    def addVertex(self, point):
        self.listOfPoints.append(point)
        return self
        
    def draw(self, tela):
        if len(self.listOfPoints) < 3:
            print("Não forma polígono. Menos de 3 vértices.")
            return
            
        if self.isFilled:
            self.scanline(tela)
        
        if self.showEdges:
            for i in range(0,len(self.listOfPoints) - 1):     
                pI = self.listOfPoints[i]
                pF = self.listOfPoints[i+1]
                line = Line(pI.x, pI.y, pF.x, pF.y, self.edgeColor)
                line.draw(tela)
            
            pI = self.listOfPoints[-1]
            pF = self.listOfPoints[0]
            line = Line(pI.x, pI.y, pF.x, pF.y, self.edgeColor)
            line.draw(tela)
            
            for i in range(0,len(self.listOfPoints)):               
                self.listOfPoints[i].draw(tela)
    
    def scanline(self, tela): 
        yMax = self.listOfPoints[0].y
        for point in self.listOfPoints:
            if point.y > yMax:
                yMax = point.y
                
        y = yMax
        
        edgeTable = []
        for i in range (0, yMax+1):
            edgeTable.append([])
            
        for i in range(0,len(self.listOfPoints) - 1):     
            
            if self.listOfPoints[i].y - self.listOfPoints[i+1].y != 0:
                edge = EdgeInfo(self.listOfPoints[i], self.listOfPoints[i+1])
                yMin = edge.yMin
                if yMin < y:
                    y = yMin
    
                edgeTable[yMin].append(edge)       
        
        if self.listOfPoints[-1].y - self.listOfPoints[0].y != 0:
            edge = EdgeInfo(self.listOfPoints[-1], self.listOfPoints[0])
            yMin = edge.yMin
            if yMin < y:
                y = yMin
            edgeTable[yMin].append(edge)
      
        active_et = []
        
        while y <= yMax:
                    
            active_et.extend(edgeTable[y])
            edgeTable[y] = []
            active_et.sort(key = sortByX)
            
            for i in range(0, len(active_et) - 1, 2):
                for x in range(int(active_et[i].x), int(active_et[i + 1].x + 1)):
                    tela.definirPixel(x, y, self.fillColor)
            y = y + 1
            
            delL = []
            for point in active_et:
                if point.yMax <= y:
                    delL.append(point)
            
            for point in delL:
                active_et.remove(point)
            
            delL.clear()
            
            for point in active_et:
                point.updateX()
                
                
    def empty(self, ET):
        for point in ET:
            if point:
                return False
        
        return True

def sortByX(point):
    return point.x

In [7]:
class Preenchimento(Polygon):
    def __init__(self, showEdges, edgeColor, isFilled, fillColor, padrao):
        super().__init__(showEdges, edgeColor, isFilled, fillColor)
        self.padrao = padrao
        
    def scanline(self, tela): 
        yMax = self.listOfPoints[0].y
        for point in self.listOfPoints:
            if point.y > yMax:
                yMax = point.y
                
        y = yMax 
        
        edge_table = [[] for _ in range(yMax + 1)]
            
        for i in range(0,len(self.listOfPoints) - 1):     
            if self.listOfPoints[i].y - self.listOfPoints[i+1].y != 0:
                edge = EdgeInfo(self.listOfPoints[i], self.listOfPoints[i+1])
                yMin = edge.yMin
                if yMin < y:
                    y = yMin
    
                edge_table[yMin].append(edge)       
        
        if self.listOfPoints[-1].y - self.listOfPoints[0].y != 0:
            edge = EdgeInfo(self.listOfPoints[-1], self.listOfPoints[0])
            yMin = edge.yMin
            if yMin < y:
                y = yMin
            edge_table[yMin].append(edge)
      
        active_et = []
        
        while y <= yMax:

            active_et.extend(edge_table[y])
            edge_table[y] = []
            active_et.sort(key = sortByX)
            
            for i in range(0, len(active_et) - 1, 2):
                for x in range(int(active_et[i].x), int(active_et[i + 1].x + 1)):
                    padraoAtual = self.padrao[y % len(self.padrao)]
                    pintar = padraoAtual[x % len(padraoAtual)] == 1
                    if (pintar):tela.definirPixel(x, y, self.fillColor)
            
            y += 1
            
            del_list = [point for point in active_et if point.yMax <= y]
            for point in del_list:
                active_et.remove(point)
            
            del_list.clear()
            
            for point in active_et:
                point.updateX()

In [8]:
tela = Tela("Exemplo 1", pygame.Color(255, 255, 255, 255), 200, 200)

polygon = Preenchimento(True, pygame.Color(255, 0, 0, 255), True, pygame.Color(0, 0, 255, 255), [[0,1,1,1,0], [1,0,0,0,1]])
polygon.addVertex(Point2D(30, 40, pygame.Color(0)))
polygon.addVertex(Point2D(50, 60, pygame.Color(0)))
polygon.addVertex(Point2D(90, 50, pygame.Color(0)))
polygon.addVertex(Point2D(120, 80, pygame.Color(0)))
polygon.addVertex(Point2D(50, 110, pygame.Color(0)))

tela.executar(polygon)

In [9]:
tela = Tela("Exemplo 2", pygame.Color(255, 255, 255, 255), 200, 200)

polygon = Preenchimento(True, pygame.Color(255, 255, 0, 255), True, pygame.Color(0, 0, 255, 255), [[0,1,1,0], [0,1,1,0]])
polygon.addVertex(Point2D(20, 30, pygame.Color(0)))
polygon.addVertex(Point2D(70, 10, pygame.Color(0)))
polygon.addVertex(Point2D(130, 50, pygame.Color(0)))
polygon.addVertex(Point2D(130, 100, pygame.Color(0)))
polygon.addVertex(Point2D(70, 70, pygame.Color(0)))
polygon.addVertex(Point2D(20, 90, pygame.Color(0)))

tela.executar(polygon)

In [10]:
tela = Tela("Exemplo 3", pygame.Color(255, 255, 255, 255), 200, 200)

polygon = Preenchimento(True, pygame.Color(255, 0, 255, 255), True, pygame.Color(0, 0, 255, 255), [[1,0], [0,1]])
polygon.addVertex(Point2D(10, 10, pygame.Color(0)))
polygon.addVertex(Point2D(10, 70, pygame.Color(0)))
polygon.addVertex(Point2D(50, 30, pygame.Color(0)))
polygon.addVertex(Point2D(80, 60, pygame.Color(0)))
polygon.addVertex(Point2D(80, 10, pygame.Color(0)))

tela.executar(polygon)